Prompts que darão instruções para o comportamento da IA

In [ ]:
RESPONSE_TEMPLATE = """\
You are an expert programmer and problem-solver, tasked with answering any question \
about StackSpot AI.

Generate a comprehensive and informative answer of 80 words or less for the \
given question based solely on the provided search results (URL and content). You must \
only use information from the provided search results. Use an unbiased and \
journalistic tone. Combine search results together into a coherent answer. Do not \
repeat text. Cite search results using [${{number}}] notation. Only cite the most \
relevant results that answer the question accurately. Place these citations at the end \
of the sentence or paragraph that reference them - do not put them all at the end. If \
different results refer to different entities within the same name, write separate \
answers for each entity.

You should use bullet points in your answer for readability. Put citations where they apply
rather than putting them all at the end.

If there is nothing in the context relevant to the question at hand, just say "Hmm, \
I'm not sure." Don't try to make up an answer.

Anything between the following `context`  html blocks is retrieved from a knowledge \
bank, not part of the conversation with the user. 

<context>
    {context} 
<context/>

REMEMBER: If there is no relevant information within the context, just say "Hmm, I'm \
not sure." Don't try to make up an answer. Anything between the preceding 'context' \
html blocks is retrieved from a knowledge bank, not part of the conversation with the \
user.\
"""

REPHRASE_TEMPLATE = """\
Given the following conversation and a follow up question, rephrase the follow up \
question to be a standalone question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone Question:"""

Classe modelo que será utilizada nas chamadas para IA

In [ ]:
from typing import Dict, List, Optional
from langchain_core.pydantic_v1 import BaseModel

class ChatRequest(BaseModel):
    question: str
    chat_history: Optional[List[Dict[str, str]]]

Método responsável pela criação do recuperador do vectorstore

In [ ]:
import weaviate

from langchain_community.vectorstores import Weaviate
from langchain_core.retrievers import BaseRetriever
from langchain_openai import OpenAIEmbeddings
from langchain_core.embeddings import Embeddings

WEAVIATE_DOCS_INDEX_NAME = 'My_Docs_Index_Name'
WEAVIATE_URL = 'http://localhost:8080'

def get_embeddings_model() -> Embeddings:
    return OpenAIEmbeddings(model="text-embedding-3-small", chunk_size=200)

def get_retriever() -> BaseRetriever:
    client = weaviate.Client(
        url=WEAVIATE_URL
    )
    weaviate_client =  Weaviate(
        client=client,
        index_name=WEAVIATE_DOCS_INDEX_NAME,
        text_key="text",
        embedding=get_embeddings_model(),
        by_text=False,
        attributes=["source", "title"],
    )
    return weaviate_client.as_retriever(search_kwargs=dict(k=6))

Método responsável pela criação da chain com a LLM e o retriever usando o prompt REPHRASE_TEMPLATE que criamos

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.language_models import BaseLanguageModel
from langchain_core.runnables import (
    Runnable,
    RunnableBranch,
    RunnableLambda
)
from langchain_core.prompts import PromptTemplate

def create_retriever_chain(
    llm: BaseLanguageModel, retriever: BaseRetriever
) -> Runnable:
    CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(REPHRASE_TEMPLATE)
    condense_question_chain = (
        CONDENSE_QUESTION_PROMPT | llm | StrOutputParser()
    ).with_config(
        run_name="CondenseQuestion",
    )
    conversation_chain = condense_question_chain | retriever
    return RunnableBranch(
        (
            RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
                run_name="HasChatHistoryCheck"
            ),
            conversation_chain.with_config(run_name="RetrievalChainWithHistory"),
        ),
        (
            RunnableLambda(itemgetter("question")).with_config(
                run_name="Itemgetter:question"
            )
            | retriever
        ).with_config(run_name="RetrievalChainWithNoHistory"),
    ).with_config(run_name="RouteDependingOnChatHistory")


Método responsável pela formatação dos resultados encontrados no vectorstore para o formato esperado pela IA

In [ ]:
from typing import Sequence

from langchain_core.documents import Document

def format_docs(docs: Sequence[Document]) -> str:
    formatted_docs = []
    for i, doc in enumerate(docs):
        doc_string = f"<doc id='{i}'>{doc.page_content}</doc>"
        formatted_docs.append(doc_string)
    return  "\n".join(formatted_docs)

Método responsável por montar o histórico da conversa no modelo esperado pela IA

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

def serialize_history(request: ChatRequest):
    chat_history = request["chat_history"] or []
    converted_chat_history = []
    for message in chat_history:
        if message.get("human") is not None:
            converted_chat_history.append(HumanMessage(content=message["human"]))
        if message.get("ai") is not None:
            converted_chat_history.append(AIMessage(content=message["ai"]))
    return converted_chat_history

Método responsável pela criação da chain que irá orquestrar a busca no vectorstore, definir os contextos, separar o histórico e normalizar a respota da IA

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables import RunnableMap

def create_chain(
    llm: BaseLanguageModel,
    retriever: BaseRetriever,
) -> Runnable:
    retriever_chain = create_retriever_chain(
        llm,
        retriever,
    ).with_config(run_name="FindDocs")
    _context = RunnableMap(
        {
            "context": retriever_chain | format_docs,
            "question": itemgetter("question"),
            "chat_history": itemgetter("chat_history"),
        }
    ).with_config(run_name="RetrieveDocs")
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", RESPONSE_TEMPLATE),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{question}"),
        ]
    )

    response_synthesizer = (prompt | llm | StrOutputParser()).with_config(
        run_name="GenerateResponse",
    )
    return (
        {
            "question": RunnableLambda(itemgetter("question")).with_config(
                run_name="Itemgetter:question"
            ),
            "chat_history": RunnableLambda(serialize_history).with_config(
                run_name="SerializeHistory"
            ),
        }
        | _context
        | response_synthesizer
    )

Criação do array que armazenará o input do histórico do chat, criação da LLM e chamada dos métodos get_retriever e create_chain

In [ ]:
from langchain_openai import ChatOpenAI

chat_history = []

llm = ChatOpenAI(
    model="gpt-3.5-turbo-1106",
    streaming=True,
    temperature=0,
    verbose=True
)

retriever = get_retriever()
answer_chain = create_chain(
    llm,
    retriever,
)

Método responsável por fazer as chamadas para IA passando as perguntas do usuário e guardar o input de histórico do chat

In [ ]:
def call_ai(input: str):
    response = answer_chain.invoke({'question': input, 'chat_history': chat_history})
    chat_history.append({'human': input, 'ai': response})
    return response

In [ ]:
call_ai("O que é stackspot ia?")